In [1]:
import pandas as pd

from utils import fetch_handbook_dataframe
import pathlib
import sqlite3
import geopandas as gpd

data_dir = pathlib.Path("..").resolve() / "data"
gpkg = str(data_dir / "ppm.gpkg")
# Create connection and load spatialite extension
df = gpd.read_file(gpkg, layer="ministers")
aph_df = fetch_handbook_dataframe(df)
aph_df

,PHID,GivenName,MiddleNames,FamilyName,PreferredName,DisplayName,DateOfBirth,DateOfDeath,PlaceOfBirth,PlaceOfDeath,...,RepresentedMinistries,RepresentedShadowMinistries,ParliamentaryPositions,Honours,Occupations,SecondaryOccupations,Qualifications,ElectorateService,PartyParliamentaryService,PartyCommitteeService
0,300147,Aaron,,VIOLI,,"VIOLI, Aaron",,,,,...,[],[],[],[],[],[],[],"[{'Electorate': 'Casey', 'State': 'Victoria', ...","[{'RoSId': 59490, 'RoSType': 'Parliamentary Se...","[{'RoSId': 60048, 'RoSType': 'Committee Servic..."
0,M3C,Adam,Paul,BANDT,,"BANDT, Dr Adam Paul",1972-03-11,,Adelaide,,...,[],[],[],[],"[Barrister from 2008 to 2010., Research office...","[Solicitor, Social Professionals nfd, Barriste...","[Doctor of Philosophy, Monash University, Bach...","[{'Electorate': 'Melbourne', 'State': 'Victori...","[{'RoSId': 27172, 'RoSType': 'Parliamentary Se...","[{'RoSId': 1164, 'RoSType': 'Committee Service..."
0,269375,Alexander,Charles,ANTIC,(Alex),"ANTIC, Alexander (Alex) Charles",1974-12-17,,Adelaide,,...,[],[],[],[],[Solicitor from 2002 to 2019.],"[Solicitor, Legal, Social and Welfare Professi...","[Bachelor of Laws, University of Adelaide, Bac...",[],"[{'RoSId': 54265, 'RoSType': 'Parliamentary Se...","[{'RoSId': 54358, 'RoSType': 'Committee Servic..."
0,HWO,Alexander,George,HAWKE,(Alex),"HAWKE, the Hon. Alexander (Alex) George",1977-07-09,,Wollongong,,...,"[Assistant Minister, Assistant Minister, Assis...",[],[Member of the Speaker's Panel],[],"[Adviser to R Williams, MLA, 2007., Adviser to...","[Retail Manager (General), Hospitality, Retail...","[Bachelor of Arts (Government and Philosophy),...","[{'Electorate': 'Mitchell', 'State': 'New Sout...","[{'RoSId': 27382, 'RoSType': 'Parliamentary Se...","[{'RoSId': 1508, 'RoSType': 'Committee Service..."
0,144732,Alicia,Emma,PAYNE,,"PAYNE, Alicia Emma",1982-07-24,,Canberra,,...,[],[],[],[],"[Chief of Staff to Hon. J Macklin MP, Shadow M...","[Policy Analyst, Business, Human Resource and ...","[Bachelor of Economics Honours, University of ...","[{'Electorate': 'Canberra', 'State': 'Australi...","[{'RoSId': 54199, 'RoSType': 'Parliamentary Se...","[{'RoSId': 54738, 'RoSType': 'Committee Servic..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,288713,Sam,,BIRRELL,,"BIRRELL, Sam",,,,,...,[],[],[],[],[],[],[],"[{'Electorate': 'Nicholls', 'State': 'Victoria...","[{'RoSId': 59510, 'RoSType': 'Parliamentary Se...","[{'RoSId': 60028, 'RoSType': 'Committee Servic..."
0,300130,Sam,,LIM,,"LIM, Sam",1961-06-06,,"Parit Zing, Muar, Johor",,...,[],[],[],"[Police Officer of the Year, 2020.]","[Police Officer from 6.11.2006 to 20.5.2022., ...","[Security Officers and Guards nec, Police Offi...","[Diploma in Public Safety (Policing), Western ...","[{'Electorate': 'Tangney', 'State': 'Western A...","[{'RoSId': 59223, 'RoSType': 'Parliamentary Se...","[{'RoSId': 59842, 'RoSType': 'Committee Servic..."
0,300122,Samuel,Thomas,RAE,(Sam),"RAE, Samuel (Sam) Thomas",1986-09-26,,Canberra,,...,[],[],[],[],"[Partner at PwC from 2019 to 2021., State Secr...","[Call or Contact Centre Operator, Child Care W...","[Master of Business Administration, Melbourne ...","[{'Electorate': 'Hawke', 'State': 'Victoria', ...","[{'RoSId': 59317, 'RoSType': 'Parliamentary Se...","[{'RoSId': 59813, 'RoSType': 'Committee Servic..."
0,I0U,Sarah,Coral,HANSON-YOUNG,,"HANSON-YOUNG, Sarah Coral",1981-12-23,,Melbourne,,...,[],[],[],[],"[Freelance writer and columnist from 2007., Ba...","[Bank Worker, Sales Assistant (General), Journ...","[Bachelor of Social Science, University of Ade...",[],"[{'RoSId': 27831, 'RoSType': 'Parliamentary Se...","[{'RoSId': 3191, 'RoSType': 'Committee Service..."


In [7]:

ministers_table = aph_df[["PHID", "GivenName", "MiddleNames", "FamilyName", "PreferredName", "DisplayName","DateOfBirth", "PlaceOfBirth", "Image", "Gender", "MaritalStatus", "CountryOfBirth", "StateOfBirth", "Party", "PartyAbbrev", "SenateState", "State", "StateAbbrev", "InCurrentParliament", "ServiceHistory_Start", "ServiceHistory_End", "ServiceHistory_Days", "ElectedMemberNo", "ElectedSenatorNo", "FirstNations"]]
ministers_occupations = aph_df[["PHID", "Occupations"]].explode("Occupations")
ministers_secondary_occupations = aph_df[["PHID", "SecondaryOccupations"]].explode("SecondaryOccupations")
ministers_qualifications = aph_df[["PHID", "Qualifications"]].explode("Qualifications")

In [10]:
db_con = sqlite3.connect(str(data_dir / "ppm.gpkg"))
ministers_table.to_sql("ministers_aph", db_con, if_exists="replace")
ministers_occupations.dropna().to_sql("ministers_occupations", db_con, if_exists="replace", index_label="fid")
ministers_secondary_occupations.dropna().to_sql("ministers_secondary_occupations", db_con, if_exists="replace", index_label="fid")
ministers_qualifications.dropna().to_sql("ministers_secondary_qualifications", db_con, if_exists="replace", index_label="fid")
aph_df.to_parquet(data_dir/"aph_all.parquet")

361

In [14]:
import geopandas as gpd
aus_schools = gpd.read_file(data_dir / "school-location-2022.csv")

In [19]:
aus_schools["geometry"] = gpd.points_from_xy(aus_schools.Longitude, aus_schools.Latitude)
aus_schools.set_crs("EPSG:4326", inplace=True)

,Calendar Year,ACARA SML ID,Location AGE ID,School AGE ID,School Name,Suburb,State,Postcode,School Sector,School Type,...,Statistical Area 3 Name,Statistical Area 4,Statistical Area 4 Name,Local Government Area,Local Government Area Name,State Electoral Division,State Electoral Division Name,Commonwealth Electoral Division,Commonwealth Electoral Division Name,geometry
0,2022,42689,50438,10438,Albury High School,ALBURY,NSW,2640,Government,Secondary,...,Albury,109,Murray,10050,Albury,10001,Albury,114,Farrer,POINT (146.91544 -36.07285)
1,2022,42648,50444,10444,Table Top Public School,TABLE TOP,NSW,2640,Government,Primary,...,Albury,109,Murray,10050,Albury,10001,Albury,114,Farrer,POINT (147.00708 -35.96794)
2,2022,42654,50440,10440,Albury North Public School,ALBURY,NSW,2640,Government,Primary,...,Albury,109,Murray,10050,Albury,10001,Albury,114,Farrer,POINT (146.93240 -36.06418)
3,2022,43599,41573,1573,St Anne's Primary School,NORTH ALBURY,NSW,2640,Catholic,Primary,...,Albury,109,Murray,10050,Albury,10001,Albury,114,Farrer,POINT (146.93611 -36.06032)
4,2022,43628,57922,17922,Xavier High School,NORTH ALBURY,NSW,2640,Catholic,Secondary,...,Albury,109,Murray,10050,Albury,10001,Albury,114,Farrer,POINT (146.93513 -36.06267)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10679,2022,49894,50079,10079,Jervis Bay School,JERVIS BAY,ACT,2540,Government,Primary,...,Jervis Bay,901,Other Territories,99399,Unincorp. Other Territories,99191,Unclassified (OT),803,Fenner,POINT (150.70557 -35.13154)
10680,2022,48241,55172,15172,Christmas Island District High School,CHRISTMAS ISLAND,WA,6798,Government,Combined,...,Christmas Island,901,Other Territories,99399,Unincorp. Other Territories,99191,Unclassified (OT),701,Lingiari,POINT (105.66860 -10.43520)
10681,2022,48244,55176,15176,Cocos Islands District High School,COCOS ISLAND,WA,6799,Government,Combined,...,Cocos (Keeling) Islands,901,Other Territories,99399,Unincorp. Other Territories,99191,Unclassified (OT),701,Lingiari,POINT (96.82880 -12.18800)
10682,2022,52748,87102,15176,Cocos Islands District High School - Home Isla...,COCOS ISLAND,WA,6799,Government,Combined,...,Cocos (Keeling) Islands,901,Other Territories,99399,Unincorp. Other Territories,99191,Unclassified (OT),701,Lingiari,POINT (96.82880 -12.18800)


In [20]:
df = gpd.read_file(gpkg, layer="education")

In [29]:
df.is_high_school.count()

281

In [24]:
merged = pd.merge(df, aus_schools, how="inner", left_on=df.school_name, right_on=aus_schools["School Name"] )

In [27]:
merged[merged.geometry_x.buffer(0.01).intersects(merged.geometry_y)]

/tmp/ipykernel_279990/4060953232.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged[merged.geometry_x.buffer(0.01).intersects(merged.geometry_y)]


,key_0,ogc_fid,school_name,school_link,is_university,is_high_school,is_alt_edu,operational_status,wkt,is_public,...,Statistical Area 3 Name,Statistical Area 4,Statistical Area 4 Name,Local Government Area,Local Government Area Name,State Electoral Division,State Electoral Division Name,Commonwealth Electoral Division,Commonwealth Electoral Division Name,geometry_y
0,Anglican Church Grammar School,4.0,Anglican Church Grammar School,http://www.wikidata.org/entity/Q4763439,0,1,0,NaN,Point(153.054 -27.4819),NaN,...,Brisbane Inner - East,305,Brisbane Inner City,31000,Brisbane,30076,South Brisbane,313,Griffith,POINT (153.05093 -27.48221)
1,Aquinas College,5.0,Aquinas College,http://www.wikidata.org/entity/Q4782971,0,1,0,NaN,Point(145.235555555 -37.823055555),NaN,...,Maroondah,211,Melbourne - Outer East,24410,Maroondah,27101,Ringwood (Eastern Metropolitan),211,Deakin,POINT (145.23518 -37.82303)
4,Armadale Senior High School,6.0,Armadale Senior High School,http://www.wikidata.org/entity/Q4792525,0,1,0,NaN,Point(116.015 -32.1617),NaN,...,Armadale,506,Perth - South East,50210,Armadale,50202,Armadale (East Metropolitan),502,Burt,POINT (116.01635 -32.16163)
5,Brisbane State High School,7.0,Brisbane State High School,http://www.wikidata.org/entity/Q4968616,0,1,0,NaN,Point(153.018 -27.4803),NaN,...,Brisbane Inner,305,Brisbane Inner City,31000,Brisbane,30076,South Brisbane,313,Griffith,POINT (153.01874 -27.47935)
6,Bundamba State Secondary College,8.0,Bundamba State Secondary College,http://www.wikidata.org/entity/Q4997498,0,1,0,NaN,Point(152.813 -27.6147),NaN,...,Ipswich Inner,310,Ipswich,33960,Ipswich,30010,Bundamba,301,Blair,POINT (152.81370 -27.61490)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Trinity Grammar School,216.0,Trinity Grammar School,https://www.wikidata.org/wiki/Q7842815,0,1,0,NaN,Point(151.130556 -33.899444),NaN,...,Strathfield - Burwood - Ashfield,120,Sydney - Inner West,14170,Inner West,10078,Summer Hill,144,Watson,POINT (151.13071 -33.89855)
155,Underdale High School,238.0,Underdale High School,http://www.wikidata.org/entity/Q7883557,0,1,0,NaN,Point(138.544674 -34.9189195),NaN,...,West Torrens,404,Adelaide - West,48410,West Torrens,40046,West Torrens,405,Hindmarsh,POINT (138.54417 -34.91895)
156,Westfields Sports High School,217.0,Westfields Sports High School,http://www.wikidata.org/entity/Q7988785,0,1,0,NaN,Point(150.9199898 -33.867973),NaN,...,Fairfield,127,Sydney - South West,12850,Fairfield,10029,Fairfield,128,McMahon,POINT (150.92006 -33.86835)
157,Wynyard High School,186.0,Wynyard High School,http://www.wikidata.org/entity/Q916063,0,1,0,NaN,Point(145.7169952 -40.9868049),NaN,...,Burnie - Ulverstone,604,West and North West,65410,Waratah-Wynyard,60209,Braddon (Murchison),602,Braddon,POINT (145.71764 -40.98627)


In [28]:
matched_schools = merged[merged.geometry_x.buffer(0.01).intersects(merged.geometry_y)]
# matched_schools.to_sql("secondary_schools", db_con, if_exists="replace")


/tmp/ipykernel_279990/2759202524.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  matched_schools = merged[merged.geometry_x.buffer(0.01).intersects(merged.geometry_y)]


In [34]:
cleaned_matched_schools = matched_schools[["school_name", "school_link", "operational_status",  "ACARA SML ID", "Suburb", "State", "Postcode", "School Sector", "School Type", "ABS Remoteness Area", "ABS Remoteness Area Name", "Meshblock", "Statistical Area 1", "Statistical Area 2", "Statistical Area 2 Name", "Statistical Area 3", "Statistical Area 3 Name", "Statistical Area 4", "Statistical Area 4 Name", "Local Government Area", "Local Government Area Name", "State Electoral Division", "State Electoral Division Name", "Commonwealth Electoral Division", "Commonwealth Electoral Division Name", "geometry_x"]]
cleaned_matched_schools.set_geometry("geometry_x", inplace=True)
cleaned_matched_schools.rename_geometry("geometry", inplace=True)

/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.py:393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.rename(columns={geometry_col: col}, inplace=inplace)
/home/cam/.cache/pypoetry/virtualenvs/pollypedagogymap-NEXWE1VB-py3.10/lib/python3.10/site-packages/geopandas/geodataframe.p

In [39]:
cleaned_matched_schools.to_file(str(data_dir / "ppm.gpkg"), layer='secondary_schools', driver="GPKG", ifexists="replace")


RuntimeError: GDAL Error: sqlite3_exec(CREATE TABLE "secondary_schools" ( "fid" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, "geom" POINT, "school_name" TEXT, "school_link" TEXT, "operational_status" TEXT, "ACARA SML ID" TEXT, "Suburb" TEXT, "State" TEXT, "Postcode" TEXT, "School Sector" TEXT, "School Type" TEXT, "ABS Remoteness Area" TEXT, "ABS Remoteness Area Name" TEXT, "Meshblock" TEXT, "Statistical Area 1" TEXT, "Statistical Area 2" TEXT, "Statistical Area 2 Name" TEXT, "Statistical Area 3" TEXT, "Statistical Area 3 Name" TEXT, "Statistical Area 4" TEXT, "Statistical Area 4 Name" TEXT, "Local Government Area" TEXT, "Local Government Area Name" TEXT, "State Electoral Division" TEXT, "State Electoral Division Name" TEXT, "Commonwealth Electoral Division" TEXT, "Commonwealth Electoral Division Name" TEXT)) failed: table "secondary_schools" already exists. Failed to write record: <fiona.model.Feature object at 0x7f235e0df040>

In [38]:
cleaned_matched_schools.iloc[:, 25]

0      POINT (153.05400 -27.48190)
1      POINT (145.23556 -37.82306)
4      POINT (116.01500 -32.16170)
5      POINT (153.01800 -27.48030)
6      POINT (152.81300 -27.61470)
                  ...             
154    POINT (151.13056 -33.89944)
155    POINT (138.54467 -34.91892)
156    POINT (150.91999 -33.86797)
157    POINT (145.71700 -40.98680)
158    POINT (116.76017 -31.89322)
Name: geometry, Length: 133, dtype: geometry